# Spark assignment 1: Pairs

Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

red_apple	100500

crazy_zoo	42

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

Dataset location: /data/wiki/en_articles_part

The result on the sample dataset:
narodnaya_gazeta    1
narodnaya_volya 9

In [1]:
import re
from pyspark import SparkConf, SparkContext

In [2]:
try:
    sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local").set("spark.cores.max", "16"))
except:
    pass

In [3]:
def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text.lower(), flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text.lower(), flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [4]:
def pairs(words):
    out = []
    for w1, w2 in zip(words, words[1:]):
        out.append((w1.lower() + "_" + w2.lower(), 1))
    return out

In [ ]:
#wiki_file = "/data/wiki/en_articles_part/articles-part" # coursera version
wiki_file = "/mnt/data/articles-part" # local version

result = (sc.textFile(wiki_file, 16) #need to use this on their servers: /data/wiki/en_articles_part/articles-part
        .map(parse_article)
        .flatMap(pairs)
        .reduceByKey(lambda x,y : x+y)
        .filter(lambda value: value[0][:9] == "narodnaya")
       ).collect()

In [ ]:
for key, count in result:
    print("%s\t%d" % (key, count))